
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# リカレント ニューラル ネットワーク

:ラベル: `sec_rnn`

 :numref: `sec_language-model`では、言語モデリングのためのマルコフ モデルと $n$-gram について説明しました。タイム ステップ $t$ におけるトークン $x_t$ の条件付き確率は、前のトークン $n-1$ にのみ依存します。 $x_t$ のタイムステップ $t-(n-1)$ よりも前のトークンの考えられる効果を組み込みたい場合は、$n$ を増やす必要があります。ただし、語彙セット $\mathcal{V}$ に $|\mathcal{V}|^n$ の数値を保存する必要があるため、モデル パラメーターの数も指数関数的に増加します。したがって、$P(x_t \mid x_{t-1}, \ldots, x_{t-n+1})$ をモデル化するよりも、潜在変数モデルを使用することが望ましいです。

 $$P(x_t \mid x_{t-1}, \ldots, x_1) \およそ P(x_t \mid h_{t-1}),$$

ここで、$h_{t-1}$ は、タイム ステップ $t-1$ までのシーケンス情報を保存する*隠れステート*です。一般に、任意のタイム ステップ $t$ の隠れ状態は、現在の入力 $x_{t}$ と前の隠れ状態 $h_{t-1}$ の両方に基づいて計算できます。

 $$h_t = f(x_{t}, h_{t-1}).$$ :eqlabel: `eq_ht_xt`

 :eqref: `eq_ht_xt`の十分に強力な関数 $f$ の場合、潜在変数モデルは近似ではありません。結局のところ、$h_t$ はこれまでに観測したすべてのデータを単に保存しているだけかもしれません。ただし、計算とストレージの両方のコストが高くなる可能性があります。

 :numref: `chap_perceptrons`の隠れユニットを含む隠れ層について説明したことを思い出してください。隠れ層と隠れ状態が 2 つのまったく異なる概念を参照していることは注目に値します。説明したように、非表示レイヤーは、入力から出力へのパス上で表示されないレイヤーです。隠れ状態は、技術的に言えば、特定のステップで行うことへの*入力*であり、前のタイム ステップのデータを参照することによってのみ計算できます。

*リカレント ニューラル ネットワーク*(RNN) は、隠れた状態を持つニューラル ネットワークです。 RNN モデルを紹介する前に、まず :numref: `sec_mlp`で紹介された MLP モデルを再確認します。


In [1]:
import torch
from d2l import torch as d2l


## 隠れ状態のないニューラル ネットワーク

単一の隠れ層を持つ MLP を見てみましょう。隠れ層の活性化関数を $\phi$ とします。バッチ サイズ $n$ および $d$ の入力を持つサンプル $\mathbf{X} \in \mathbb{R}^{n \times d}$ のミニバッチを指定すると、隠れ層の出力 $\mathbf{H} \in \mathbb{R}^{n \times h}$ は次のように計算されます。

 $$\mathbf{H} = \phi(\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h).$$ :eqlabel: `rnn_h_without_state`

 :eqref: `rnn_h_without_state`には、重みパラメータ $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$、バイアス パラメータ $\mathbf{b}_h \in \ があります。 mathbb{R}^{1 \times h}$、および隠れ層の隠れユニットの数 $h$。したがって、ブロードキャスト ( :numref: `subsec_broadcasting`参照) が合計中に適用されます。次に、隠れ層の出力 $\mathbf{H}$ が出力層の入力として使用されます。出力層は次のように与えられます。

 $$\mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_q,$$

ここで、 $\mathbf{O} \in \mathbb{R}^{n \times q}$ は出力変数、 $\mathbf{W}_{hq} \in \mathbb{R}^{h \times q }$ は重みパラメータ、 $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$ は出力層のバイアス パラメータです。分類問題の場合は、$\text{softmax}(\mathbf{O})$ を使用して出力カテゴリの確率分布を計算できます。

これは、以前に :numref: `sec_sequence`で解決した回帰問題と完全に類似しているため、詳細は省略します。特徴とラベルのペアをランダムに選択し、自動微分と確率的勾配降下によってネットワークのパラメーターを学習できると言えば十分です。

## 隠れた状態を持つリカレント ニューラル ネットワーク

:label: `subsec_rnn_w_hidden_states`

隠れた状態がある場合、問題はまったく異なります。もう少し詳しく構造を見てみましょう。

*タイムステップ $t$ における入力 $\mathbf{X} t \in \mathbb{R}^{n \times d}$ のミニバッチがあると仮定します。言い換えれば、$n$ シーケンス例のミニバッチの場合、$\mathbf{X}_t$ の各行はシーケンスからのタイム ステップ $t$ における 1 つのサンプルに対応します。次に、タイムステップ $t$ の隠れ層出力を $\mathbf{H}_t \in \mathbb{R}^{n \times h}$ で表します。 MLP とは異なり、ここでは前のタイムステップからの隠れ層出力 $\mathbf{H} {t-1}$ を保存し*、新しい重みパラメーター $\mathbf{W}_{hh} \in \mathbb{R を導入します。 }^{h \times h}$ は、前のタイム ステップの隠れ層出力を現在のタイム ステップで使用する方法を記述します。具体的には、現在のタイム ステップの隠れ層出力の計算は、現在のタイム ステップの入力と前のタイム ステップの隠れ層の出力によって決定されます。

 $$\mathbf{H} *t = \phi(\mathbf{X}_t \mathbf{W}* {xh} + \mathbf{H} *{t-1} \mathbf{W}* {hh} + \mathbf{b }_h).$$ :eqlabel: `rnn_h_with_state`

 :eqref: `rnn_h_without_state`と比較すると、 :eqref: `rnn_h_with_state`項 $\mathbf{H} *{t-1} \mathbf{W}* {hh}$ を 1 つ追加し、 :eqref: `eq_ht_xt`をインスタンス化します。隣接するタイム ステップの隠れ層出力 $\mathbf{H} *t$ と $\mathbf{H}* {t-1}$ の間の関係から、これらの変数が現在時刻までのシーケンスの履歴情報をキャプチャして保持していることがわかります。ニューラル ネットワークの現在のタイム ステップの状態またはメモリと同様です。したがって、このような隠れ層の出力は*隠れ状態*と呼ばれます。非表示状態では現在のタイム ステップで前のタイム ステップと同じ定義が使用されるため、 :eqref: `rnn_h_with_state`の計算は*再帰的*です。したがって、前述したように、再帰的計算に基づいた隠れ状態を持つニューラル ネットワークは、*再帰的ニューラル ネットワーク*と呼ばれます。 RNN で :eqref: `rnn_h_with_state`の計算を実行する層は、*リカレント層*と呼ばれます。

 RNN を構築するにはさまざまな方法があります。 :eqref: `rnn_h_with_state`で定義された隠れ状態を持つ RNN は非常に一般的です。タイム ステップ $t$ の場合、出力層の出力は MLP の計算と似ています。

 $$\mathbf{O} *t = \mathbf{H}_t \mathbf{W}* {hq} + \mathbf{b}_q.$$

 RNN のパラメーターには、重み $\mathbf{ *W} {xh} \in \mathbb{R}^{d \times h}、\mathbf{W} {* hh} \in \mathbb{R}^{h \ が含まれます。 h}$ の乗算*、および隠れ層のバイアス $\mathbf{b} h \in \mathbb{R}^{1 \times h}$ と重み $\mathbf{W} {* hq} \in \mathbb{R}^{h \times q}$ と出力層の \mathbb{R}^{1 \times q}$ のバイアス $\mathbf{b}_q \in異なるタイム ステップであっても、RNN は常にこれらのモデル パラメーターを使用することに注意してください。したがって、RNN のパラメータ化コストは、タイム ステップ数が増加しても増加しません。

 :numref: `fig_rnn` 3 つの隣接するタイム ステップでの RNN の計算ロジックを示しています。任意のタイム ステップ $t$ で、隠れ状態の計算は次のように扱うことができます: (i)*現在のタイム ステップ $t$ での入力 $\mathbf{X} t$ と隠れ状態 $\mathbf{H を*連結します。前のタイムステップ $t *-* 1$ での {t-1}$; (ii) 連結結果を活性化関数 $\phi$ を使用して完全に接続された層に入力します。このような全結合層の出力は*、現在のタイム ステップ $t$ の隠れ状態 $\mathbf{H} t$ です。この場合、モデル パラメーターは $\mathbf{W}* {xh}$ と $\mathbf{W} *{hh}$ の連結、および $\mathbf{b}_h$ のバイアスであり、すべて :eqref から取得されます。 : `rnn_h_with_state` 。現在のタイム ステップ $t$ の隠れ状態 $\mathbf{H}_t$ は、次のタイム ステップ $t+1$ の隠れ状態 $\mathbf{H} {t+1}$ の計算に参加します。* 。さらに、$\mathbf{H}_t$ は完全に接続された出力層にも供給され、現在のタイム ステップ $t$ の出力 $\mathbf{O}_t$ が計算されます。

![](http://d2l.ai/_images/rnn.svg) :ラベル: `fig_rnn`

隠れ状態の $\mathbf{X} *t \mathbf{W}* {xh} + \mathbf{H} *{t-1} \mathbf{W}* {hh}$ の計算は行列と同等であると述べました。 $\mathbf{X} *t$ と $\mathbf{H}* {t-1}$ の連結と $\mathbf{W} *{xh}$ と $\mathbf{W}* {hh}$ の連結の乗算。これは数学で証明できますが、以下ではこれを示すために単純なコード スニペットを使用します。まず、行列`X` 、 `W_xh` 、 `H` 、および`W_hh`を定義します。その形状はそれぞれ (3, 1)、(1, 4)、(3, 4)、および (4, 4) です。 `X`に`W_xh` 、 `H`に`W_hh`をそれぞれ乗算し、これら 2 つの乗算を加算すると、形状 (3, 4) の行列が得られます。


In [2]:
X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[-1.6464, -8.4141,  1.5096,  3.9953],
        [-1.2590, -0.2353,  2.5025,  0.2107],
        [-2.5954,  0.8102, -1.3280, -1.1265]])


次に、行列`X`と`H`列 (軸 1) に沿って連結し、行列`W_xh`と`W_hh`を行 (軸 0) に沿って連結します。これら 2 つの連結により、それぞれ形状 (3, 5) と形状 (5, 4) の行列が生成されます。これら 2 つの連結された行列を乗算すると、上記と同じ形状 (3, 4) の出力行列が得られます。


In [3]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[-1.6464, -8.4141,  1.5096,  3.9953],
        [-1.2590, -0.2353,  2.5025,  0.2107],
        [-2.5954,  0.8102, -1.3280, -1.1265]])


## RNN ベースの文字レベル言語モデル

:numref: `sec_language-model`の言語モデリングでは、現在および過去のトークンに基づいて次のトークンを予測することを目的としていることを思い出してください。そのため、元のシーケンスをターゲット (ラベル) として 1 トークンずつシフトします。 :citet: `Bengio.Ducharme.Vincent.ea.2003` 、言語モデリングにニューラル ネットワークを使用することを最初に提案しました。以下では、RNN を使用して言語モデルを構築する方法を説明します。ミニバッチ サイズを 1、テキストのシーケンスを「machine」とします。後続のセクションのトレーニングを簡素化するために、テキストを単語ではなく文字にトークン化し、*文字レベルの言語モデル*を検討します。 :numref: `fig_rnn_train`文字レベル言語モデリングの RNN を介して、現在および前の文字に基づいて次の文字を予測する方法を示します。 

![](../img/rnn-train.svg) :ラベル: `fig_rnn_train`

トレーニング プロセス中に、タイム ステップごとに出力層からの出力に対してソフトマックス演算を実行し、クロスエントロピー損失を使用してモデル出力とターゲット間の誤差を計算します。隠れ層での隠れ状態の反復計算により、 :numref: `fig_rnn_train`のタイム ステップ 3 の出力、 $\mathbf{O}_3$ は、テキスト シーケンス "m"、"a"、および「c」。トレーニング データ内のシーケンスの次の文字は「h」であるため、時間損失ステップ 3 は、特徴シーケンス「m」、「a」、「c」、および「m」に基づいて生成された次の文字の確率分布に依存します。このタイムステップのターゲット「h」。

実際には、各トークンは $d$ 次元ベクトルで表され、バッチ サイズ $n&gt;1$ が使用されます。したがって、タイム ステップ $t$ での入力 $\mathbf X_t$ は、 $n\times d$ 行列になります。これは、 :numref: `subsec_rnn_w_hidden_states`で説明したものと同じです。

次のセクションでは、文字レベル言語モデルの RNN を実装します。

## まとめ

隠れ状態の再帰的計算を使用するニューラル ネットワークは、再帰的ニューラル ネットワーク (RNN) と呼ばれます。 RNN の隠れ状態は、現在のタイム ステップまでのシーケンスの履歴情報を取得できます。反復計算では、タイム ステップ数が増加しても RNN モデル パラメーターの数は増加しません。アプリケーションに関しては、RNN を使用して文字レベルの言語モデルを作成できます。

## 演習
1. RNN を使用してテキスト シーケンス内の次の文字を予測する場合、出力に必要な次元はどれくらいでしょうか?
1. なぜ RNN は、テキスト シーケンス内の以前のすべてのトークンに基づいて、あるタイム ステップでのトークンの条件付き確率を表現できるのでしょうか?
1. 長いシーケンスを逆伝播すると勾配はどうなるでしょうか?
1. このセクションで説明されている言語モデルに関連する問題にはどのようなものがありますか?



[ディスカッション](https://discuss.d2l.ai/t/1050)
